In [52]:
# Imports
import os
from pkg_resources import resource_filename
import numpy as np
from astropy.io import fits

# ASTR 257
from astr257 import img_utils

# Personal utils
from utils import *

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams
rcParams["font.family"] = "serif"
rcParams["font.serif"] = "Times New Roman"
%matplotlib inline
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = [r'\usepackage{amsmath} \usepackage{bm} \usepackage{physics}']
%config InlineBackend.figure_format = 'retina' # For high quality figures
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
data_folder = "../data-2019-09-25-shane-joey-alex-arcelia"

In [54]:
OVERWRITE = True

## Median combine the biases, darks, flats and arcs

In [55]:
biases_combined, header_biases = crawl_and_median(data_folder, 'biases')
darks_combined, header_darks = crawl_and_median(data_folder, 'darks')
flats_combined, header_flats = crawl_and_median(data_folder, 'flats')
arcs_combined, header_arcs   = crawl_and_median(data_folder, 'arcs')

## Make the masters

In [57]:
darks_master = darks_combined - biases_combined
flats_master = flats_combined - darks_master - biases_combined

# Normalize the flat frame
flats_master_normalized = flats_master / np.median(flats_master)
foo_factor = 1.1 * np.abs(np.min(flats_master_normalized))

# Calibrate the arc
# Probably just stick to the combined arc
arcs_master  = (arcs_combined  - biases_combined - darks_master) / (flats_master_normalized + foo_factor)
arcs_foo_factor = 1.1 * np.abs(np.min(arcs_master))
arcs_master  += arcs_foo_factor

# Save the masters
try:
    fits.writeto(os.path.join(data_folder, 'masters/darks_master.fits'), 
                 darks_master, header=header_darks, overwrite=OVERWRITE)
    
    fits.writeto(os.path.join(data_folder, 'masters/biases_combined.fits'), 
                 biases_combined, header=header_biases, overwrite=OVERWRITE)
    
    fits.writeto(os.path.join(data_folder, 'masters/flats_master_normalized.fits'), 
                 flats_master_normalized, header=header_flats, overwrite=OVERWRITE)
    
    fits.writeto(os.path.join(data_folder, 'masters/arcs_combined.fits'),
                arcs_combined, header=header_arcs, overwrite=OVERWRITE)
    fits.writeto(os.path.join(data_folder, 'masters/arcs_master.fits'), 
                 arcs_master, header=header_arcs, overwrite=OVERWRITE)
except:
    print('Warning: Did not overwrite file.')